In [1]:
import pandas as pd
import numpy as np
import os
import vaex

In [85]:
df = pd.read_parquet("/Users/fahad/Desktop/preprocessed_data/processed_data31.parquet")

In [2]:
df = vaex.open('output.parquet')

In [3]:
len(df)

14616848

In [4]:
df.rename(columns={'Category':'category'}, inplace=True)

In [3]:
df.columns

Index(['date', 'rawContent', 'renderedContent', 'hashtags', 'filename',
       'filepath', 'created', 'category'],
      dtype='object')

In [87]:
df.drop(columns=["url", 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'followersCount', 'friendsCount', 'mediaCount', 'rawDescription', 'renderedDescription', 'verified', "countryCode", 'type', 'location', 'latitude', 'longitude', 'country', 'fullName', 'name'], inplace=True)

In [88]:
df.drop(columns=["bookmarkCount", 'link', 'link.indices', 'link.tcourl', 'link.text', 'link.url'], inplace=True)

In [89]:
df['filepath'].value_counts()

filepath
data/BLM_2020-05-01_2022-12-31.parquet    7967
Name: count, dtype: int64

In [53]:
df['category'] = df['filepath'].apply(lambda x: os.path.basename(os.path.dirname(x)))

In [83]:
df['category'].value_counts()

category
Graying America    80041
Name: count, dtype: int64

In [4]:
df.drop(columns=['filepath', 'filename'], inplace=True)

In [92]:
df['category'] = 'Big Cities'

In [7]:
df['county'] = df['filename'].str.extract(r'BLM_(.*?)_\d{4}-\d{2}-\d{2}_\d{4}-\d{2}-\d{2}')

In [9]:
df['county'].value_counts()

county
Wheeler OR      60006
Wallowa OR      60006
Klamath OR      60006
Daviess IN      60006
Harney OR       60006
                ...  
Jefferson IL        2
Frederick MD        2
Pulaski IN          2
Jefferson WV        2
Kalamazoo MI        2
Name: count, Length: 629, dtype: int64

In [10]:
df['county'] = df['filename'].str.replace('BLM_', '')

In [10]:
df.drop(columns=['FilePath'], inplace=True)

In [11]:
df['county'] = df['county'].str.replace(' County', '')

In [21]:
df['county'] = df['county'].str.replace(' ', '')

In [11]:
df['county'].value_counts()

county
Wheeler OR      60006
Wallowa OR      60006
Klamath OR      60006
Daviess IN      60006
Harney OR       60006
                ...  
Jefferson IL        2
Frederick MD        2
Pulaski IN          2
Jefferson WV        2
Kalamazoo MI        2
Name: count, Length: 629, dtype: int64

In [9]:
county_df = pd.read_csv('County-Type-Share.csv')

In [10]:
county_df.drop(columns=['county_cleaned', 'county_state', 'State', 'Type Number', 'FIPS', 'Type Number2', 'Key', 'New Names' ], inplace=True)

In [11]:
new_df = df.merge(county_df, how='left', left_on='county', right_on='County')

In [12]:
new_df['category'].value_counts()

category
Exurbs             6025
Graying America    1004
Name: count, dtype: int64

In [13]:
new_df.drop(columns=['county', 'County'], inplace=True)

In [23]:
new_df['county'] = new_df['category'].fillna('Big Cities', inplace=True)

In [9]:
df['Category'] = 'Big Cities'

In [10]:
df.drop(columns=['filename'], inplace=True)

In [6]:
df.to_parquet('big_cities20.parquet')